## For Local master node

In [ ]:
%%local
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
spark.sparkContext.getConf().getAll()
spark.sparkContext.version
!which pyspark

In [ ]:
%%local
from pyspark.sql import SparkSession
from sagemaker_pyspark import classpath_jars # for S3 access
#spark=SparkSession.builder.config("spark.driver.extraClassPath", ":".join(classpath_jars())).config('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0').getOrCreate()
spark=SparkSession.builder.config('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0').config("spark.driver.extraClassPath", ":".join(classpath_jars())).config('spark.jars','file:///home/ec2-user/.ivy2/jars/org.apache.spark_spark-tags_2.11-2.4.0.jar,file:///home/ec2-user/.ivy2/jars/org.apache.spark_spark-avro_2.11-2.4.0.jar,file:///home/ec2-user/.ivy2/jars/org.spark-project.spark_unused-1.0.0.jar').getOrCreate()
spark=SparkSession.builder.config('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0').getOrCreate()
# spark.conf.set('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0')
#dF = spark.read.format("avro").load("/home/ec2-user/0.avro").show()
dF = spark.read.format("avro").load("s3a://sfdmdb/SF_CASE_VW/ymd=2018*/0.avro").count()
#spark.sparkContext.getConf().getAll()
#terminfo = spark.read.parquet("s3a://qaessential/jup2_stats.terminal_info/ymd=20181101").show()

In [ ]:
%%local
dF

## For Remote clusters

In [ ]:
%%info

In [ ]:
%%configure -f
{ "conf": {"spark.jars.packages": "org.apache.spark:spark-avro_2.11:2.4.0","spark.jars":"/opt/flint.jar"} }

In [ ]:
%%configure -f
{ "conf": {"spark.jars":$spark.jars:"/opt/flint.jar"} }

In [ ]:
#spark.conf.set('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0')
spark.read.format("avro").load("s3a://sfdmdb/SF_CASE_VW/ymd=20181101/0.avro").show()

In [1]:
%pyspark
from pyspark.sql.functions import to_timestamp,from_unixtime
from ts.flint import FlintContext
flintContext = FlintContext(sqlContext)
feature = spark.read.format('avro').load("s3a://vsat-lmtd/jup*_stats/vsat_stcd/year=2019/month=2/day=1/*").select(to_timestamp(from_unixtime("collection_start_time", "yyyy-MM-dd HH")).alias('time'),'device_id','statecode','statecodeduration')
target = spark.read.format("avro").load("s3a://sfdmdb/SF_CASE_VW/ymd=20190201/0.avro").show()
#df.printSchema
vsatlmtd= flintContext.read.dataframe(df)
vsatlmtd.show()

UsageError: Line magic function `%pyspark` not found.


In [ ]:
#spark.hadoop.parquet.enable.summary-metadata false
#spark.sql.parquet.mergeSchema false
#spark.sql.parquet.filterPushdown true
#spark.sql.hive.metastorePartitionPruning true
terminfo = spark.read.parquet("s3a://qaessential/jup2_stats.terminal_info/ymd=20181101").select('year','month','day','device_id','hw_type','absolute_beam_id')
vsatlmtd = spark.read.parquet("s3a://qaessential/jup2_stats.vsat_stcd/ymd=20181101").select('year','month','day','device_id','statecode','statecodeduration')
terminfo.createOrReplaceTempView("terminfo")
vsatlmtd.createOrReplaceTempView("vsatlmtd")
vsatlmtd_e=vsatlmtd.join(terminfo,['year','month','day','device_id'])
vsatlmtd_e.explain()
vsatlmtd_e.show()

In [ ]:
%%info

In [ ]:
%%sql -o result
select * from terminfo join vsatlmtd on terminfo.year=vsatlmtd.year and terminfo.month=vsatlmtd.month and terminfo.day=vsatlmtd.day and terminfo.device_id=vsatlmtd.device_id

In [1]:
%%sh
git add .
git commit -m 'aws spark notebooks and iam policies' -a
git push origin master

[master 595a33c] aws spark notebooks and iam policies
 Committer: EC2 Default User <ec2-user@ip-172-16-90-2.ec2.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 48 files changed, 1203 insertions(+), 22139 deletions(-)
 rename R_ingest.r => .ipynb_checkpoints/R_ingest-checkpoint.r (100%)
 create mode 100644 .ipynb_checkpoints/Untitled3-checkpoint.ipynb
 create mode 100644 .ipynb_checkpoints/Untitled4-checkpoint.ipynb
 create mode 100644 .ipynb_checkpoints/Untitled5-checkpoint.ipynb
 create mode 100644 .ipynb_checkpoints/Untitled6-checkpoint.ipynb
 rename etl.ipynb => .ipynb_checkpoints/aws_deepar-checkpoint.ipynb (100%)
 rename pysp

To https://github.com/emailechiu/aws.git
   8f35fe8..595a33c  master -> master
